# Example for utilization of powerfactory-tools -- Export

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import logging
import pathlib

from powerfactory_tools.exporter import PowerFactoryExporter
from powerfactory_tools.exporter import export_powerfactory_data
from psdm.steadystate_case.case import Case as SteadystateCase
from psdm.topology.topology import Topology
from psdm.topology_case.case import Case as TopologyCase

In [ ]:
# Consider to use raw strings to avoid misinterpretation of special characters, e.g. r"dir\New Project" or r"dir\1-HV-grid".
PROJECT_NAME = "PowerFactory-Tools"  # may be also full path "dir_name\project_name"
EXPORT_PATH = pathlib.Path("export")
PF_USER_PROFILE = ""  # specification may be necessary
PYTHON_VERSION = "3.10"
STUDY_CASES = ["Base", "Outage", "Industry_Park"]

## Export using an exporter instance

In [ ]:
# The exporter generates for each study case three JSON files: topology, topology_case, steadystate_case.
# Here, only active grids will be exported.
with PowerFactoryExporter(
    project_name=PROJECT_NAME,
    powerfactory_user_profile=PF_USER_PROFILE,
    python_version=PYTHON_VERSION,
    logging_level=logging.INFO,
) as exporter:
    _project_name = PROJECT_NAME.split("\\")

    # Export each study case of the specified project and save it in a separate folder in the specified export path.
    exporter.export(
        export_path=EXPORT_PATH / pathlib.Path(_project_name[-1]),
        study_case_names=STUDY_CASES,
    )

    # As alternative, export just the study case that is active at the moment
    # exporter.export(export_path=EXPORT_PATH)  # noqa: ERA001

## Export using the export function running in a new process with default parameters

In [ ]:
ext_log_file_path = pathlib.Path("export\logfile.txt")
# As the export function is executed in a process that is terminated after execution, the PowerFactory API is also closed.
export_powerfactory_data(
    export_path=EXPORT_PATH,
    project_name=PROJECT_NAME,
    study_case_names=STUDY_CASES,
    powerfactory_user_profile=PF_USER_PROFILE,
    python_version=PYTHON_VERSION,
    logging_level=logging.INFO,
    log_file_path=ext_log_file_path,
)

## Import JSON file using PSDM schema

In [ ]:
path = pathlib.Path("grids/")
schema_class = Topology
case = "Base"
json_file_name = "topology.json"
json_file_path = path / case / (case + "_HV_9_Bus_" + json_file_name)

# Import JSON as PSDM objects
data = schema_class.from_file(json_file_path)
# Import JSON as dicts
json_dict = data.model_dump()

## [Optional] Display JSON files and PSDM schema
### Display example JSON files for study case "Industry_Park"

In [ ]:
t = Topology.from_file(EXPORT_PATH / pathlib.Path("Industry_Park") / "Industry_Park_HV_9_Bus_topology.json")
t_s = t.model_dump()
print(json.dumps(t_s, sort_keys=True, default=str, indent=2))

In [ ]:
tc = TopologyCase.from_file(EXPORT_PATH / pathlib.Path("Industry_Park") / "Industry_Park_HV_9_Bus_topology_case.json")
tc_s = tc.model_dump()
print(json.dumps(tc_s, sort_keys=True, default=str, indent=2))

In [ ]:
ssc = SteadystateCase.from_file(
    EXPORT_PATH / pathlib.Path("Industry_Park") / "Industry_Park_HV_9_Bus_steadystate_case.json",
)
ssc_s = ssc.model_dump()
print(json.dumps(ssc_s, sort_keys=True, default=str, indent=2))

### Display raw JSON schema of Power System Data Model

In [ ]:
t_s = Topology.model_json_schema()
print(json.dumps(t_s, sort_keys=True, default=str, indent=2))

In [ ]:
tc_s = TopologyCase.model_json_schema()
print(json.dumps(tc_s, sort_keys=True, default=str, indent=2))

In [ ]:
ssc_s = SteadystateCase.model_json_schema()
print(json.dumps(ssc_s, sort_keys=True, default=str, indent=2))